In [27]:
import openai
import os
import pandas as pd
import mysql.connector

In [28]:
db_config = {
    "host": "15.168.221.131",
    "user": "lab13",
    "password": "lab13",
    "database": "SNS_DB",
    "charset" : "utf8mb4"
}

# MySQL 연결 설정
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

# 테이블 목록 확인
cursor.execute("SHOW TABLES;")
tables = cursor.fetchall()
print("테이블 목록:", tables)

테이블 목록: [('analysis_results',), ('hobby_keywords',), ('phone_recommendations',), ('tbCrawled_Danawa',), ('tbCrawled_Youtube',), ('users',)]


## 논의점
1. 나이, 성별에 따른 취미 정보를 따로 모아서 나중에 우리 데이터로 쓰기
- 선택 빈도 저장, 언제 저장되었고 업데이트 되었는지도 저장
2. 닉네임, 나이, 성별이 같은 경우에 중복으로 저장해야하는지 여부
- 같은 닉네임, 나이, 성별, 취미라도 입력한 시간이 다르면 새로운 행으로 저장
3. 분석결과, 폰 기종 추천은 나이, 성별, 취미 조합에 따라 달라짐


## 전체 로직
1. 닉네임, 나이, 성별, 취미를 사용자가 입력
2. DB에서 성별, 나이, 취미 기준으로 키워드 리스트 찾기
3. 있으면 사용하고 분석결과, 폰 기종도 db에서 가져오기
4. 없으면 LLM으로 키워드 리스트 생성

## 테이블 구조
1. users
- 사용자가 닉네임, 나이, 성별 입력 -> 취미 선택 -> 저장

2. hobby_keywords
- 나이, 성별, 취미 조합이 같으면 데이터 재사용
- 그렇지 않으면, LLM 호출

3. analysis_results
- 같은 나이, 성별, 취미 조합이 없으면 데이터 분석 새롭게 생성

4. phone_recommendations
- 같은 나이, 성별, 취미 조합이 없으면 LLM 호출

In [22]:
query = """
CREATE TABLE hobby_keywords (
    hobby_id INT AUTO_INCREMENT PRIMARY KEY,
    hobby VARCHAR(50) NOT NULL,
    gender ENUM('M', 'F') NOT NULL,
    age_group ENUM('10대', '20대', '30대', '40대', '50대', '60대') NOT NULL,
    keyword_list JSON NOT NULL,
    count INT NOT NULL DEFAULT 1,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    UNIQUE (hobby, gender, age_group)  
);

CREATE TABLE users (
    user_id INT AUTO_INCREMENT PRIMARY KEY,
    hobby_id INT NOT NULL,  
    nickname VARCHAR(50) NOT NULL,
    age_group ENUM('10대', '20대', '30대', '40대', '50대', '60대') NOT NULL,
    gender ENUM('M', 'F') NOT NULL,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP, 
    FOREIGN KEY (hobby_id) REFERENCES hobby_keywords(hobby_id) ON DELETE CASCADE,
    UNIQUE (hobby_id, nickname, age_group, gender)   
);

CREATE TABLE analysis_results (
    hobby_id INT NOT NULL,  
    gender ENUM('M', 'F') NOT NULL,  
    age_group ENUM('10대', '20대', '30대', '40대', '50대', '60대') NOT NULL,  
    keyword_list JSON NOT NULL,
    freq_samsung JSON NOT NULL,
    freq_apple JSON NOT NULL,
    related_words_samsung JSON NOT NULL,
    related_words_apple JSON NOT NULL,
    sentiment_samsung FLOAT NOT NULL,
    sentiment_apple FLOAT NOT NULL,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    PRIMARY KEY (hobby_id, gender, age_group),  
    FOREIGN KEY (hobby_id) REFERENCES hobby_keywords(hobby_id) ON DELETE CASCADE
);

CREATE TABLE phone_recommendations (
    hobby_id INT NOT NULL,
    gender ENUM('M', 'F') NOT NULL, 
    age_group ENUM('10대', '20대', '30대', '40대', '50대', '60대') NOT NULL,  
    recommendation_phone JSON NOT NULL,
    recommendation_reason TEXT NOT NULL,
    PRIMARY KEY (hobby_id, gender, age_group),  
    FOREIGN KEY (hobby_id) REFERENCES hobby_keywords(hobby_id) ON DELETE CASCADE
);
"""

for result in cursor.execute(query, multi=True):
    pass
print('실행 완료')

실행 완료


In [23]:
# 테이블 목록 확인
cursor.execute("SHOW TABLES;")
tables = cursor.fetchall()
print("테이블 목록:", tables)

테이블 목록: [('analysis_results',), ('hobby_keywords',), ('phone_recommendations',), ('tbCrawled_Danawa',), ('tbCrawled_Youtube',), ('users',)]


In [24]:
# 테이블 구조
def describe_table(table_name):
    cursor.execute(f"DESCRIBE {table_name};")
    columns = cursor.fetchall()
    df = pd.DataFrame(columns, columns=["Field", "Type", "Null", "Key", "Default", "Extra"])
    return df

print("users 테이블 구조")
display(describe_table("users"))

print("hobby_keywords 테이블 구조")
display(describe_table("hobby_keywords"))

print("analysis_results 테이블 구조")
display(describe_table("analysis_results"))

print("phone_recommendations 테이블 구조")
display(describe_table("phone_recommendations"))

users 테이블 구조


,Field,Type,Null,Key,Default,Extra
0,user_id,int(11),NO,PRI,None,auto_increment
1,hobby,int(11),NO,MUL,None,
2,nickname,varchar(50),NO,,None,
3,age_group,"enum('10대','20대','30대','40대','50대','60대')",NO,,None,
4,gender,"enum('M','F')",NO,,None,
5,created_at,timestamp,NO,,CURRENT_TIMESTAMP,


hobby_keywords 테이블 구조


,Field,Type,Null,Key,Default,Extra
0,hobby_id,int(11),NO,PRI,None,auto_increment
1,hobby,varchar(50),NO,,None,
2,gender,"enum('M','F')",NO,,None,
3,age_group,"enum('10대','20대','30대','40대','50대','60대')",NO,,None,
4,keyword_list,json,NO,,None,
5,count,int(11),NO,,1,
6,created_at,timestamp,NO,,CURRENT_TIMESTAMP,
7,updated_at,timestamp,NO,,CURRENT_TIMESTAMP,on update CURRENT_TIMESTAMP


analysis_results 테이블 구조


,Field,Type,Null,Key,Default,Extra
0,hobby_id,int(11),NO,MUL,None,
1,freq_samsung,json,NO,,None,
2,freq_apple,json,NO,,None,
3,related_words_samsung,json,NO,,None,
4,related_words_apple,json,NO,,None,
5,sentiment_samsung,float,NO,,None,
6,sentiment_apple,float,NO,,None,
7,created_at,timestamp,NO,,CURRENT_TIMESTAMP,on update CURRENT_TIMESTAMP


phone_recommendations 테이블 구조


,Field,Type,Null,Key,Default,Extra
0,hobby_id,int(11),NO,PRI,None,
1,gender,"enum('M','F')",NO,PRI,None,
2,age_group,"enum('10대','20대','30대','40대','50대','60대')",NO,PRI,None,
3,recommendation_phone,json,NO,,None,
4,recommendation_reason,text,NO,,None,


In [26]:
# 테스트
# JSON 데이터는 반드시 이중 쿼트(")를 사용해야 함
query = """
INSERT INTO hobby_keywords (hobby, gender, age_group, keyword_list) 
VALUES ('사진 촬영', 'F', '20대', '["배터리", "화질", "렌즈", "초점", "용량"]');
"""
cursor.execute(query)
conn.commit()
print("샘플 데이터 추가")

샘플 데이터 추가
